# Train the model
### My approach
  
    - Load the data 
        - I Load the json file
        - Clean the data
        - remove duplicates, nan values, outliers and balance the class labels to remove bias
        - tokenize it ready for training

    - Train the model 
        - I opted for a verbose approach (see src/train.py) rather than using a trainer. It made for easier MLFlow tracking.
        
    - Evaluate
        - After each batch I log precision, recall and loss in MLFlow


The first model I trained got a maximum f1 of 0.989 on the evaluation set which led me to believe there is probably some data leakage somewhere.

### If I had more time
I'd rectify the data leakage using agglomerative clustering. I’d select a cluster or two to form the data for the test set. The remaining clusters would form the train set. That way I could be more confident there was no data leakage from train into test.



In [1]:
import sys
sys.path.append('..')
from src import train, data
from transformers import AutoModelForSequenceClassification, AutoTokenizer

/Users/leedudek/Library/Mobile Documents/com~apple~CloudDocs/life/books/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
models_to_try = ["sentence-transformers/all-MiniLM-L6-v2", "bert-base-uncased", "intfloat/e5-small", "TheBritishLibrary/bl-books-genre", "avsolatorio/GIST-Embedding-v0"]
df = data.load_data(clean=True)
df.rename(columns={'austen':'labels'}, inplace=True)

In [3]:
for model_name in models_to_try:
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)
    model.to('cpu')
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    train_loader, eval_loader, test_loader = data.split(df[['text','labels']], tokenizer=tokenizer)
    train.train(model, model_name, train_loader, eval_loader, test_loader, 'cpu', num_epochs=1)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/leedudek/Library/Mobile Documents/com~apple~CloudDocs/life/books/.venv/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 17%|█▋        | 38/225 [06:35<32:32, 10.44s/it]